In [ ]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.inquiry import Inquiry
from libs.email import Email
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from string import Template
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import time
import re
import math

import types
import traceback
import threading

import pendulum

import logging
from logging.handlers import TimedRotatingFileHandler

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
market

. storage markets.json
.\storage\markets.json was deserialized!


{'name': 'Eyelashes',
 'directory': 'E:\\glitter\\Eyelashes',
 'lid': 'eyelash@glittergroupcn.com',
 'lpwd': 'Wangzhen68970747',
 'homepage': 'https://glittereyelash.en.alibaba.com',
 'lname': 'Jeremy Wang',
 'accounts': [{'lid': 'eyelash1@glittergroupcn.com',
   'lpwd': 'Wangqingxia7696',
   'lname': 'Helen Wang',
   'disabled': True}]}

In [3]:
inquiry = Inquiry(market)
# inquiry.login()

E:\glitter\Eyelashes_config [] inquiry_tracking_ids_Jeremy.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids_Jeremy.json was deserialized!


In [ ]:
inquiry.login()

In [ ]:
def is_auto_reply_needed(self, enquiry):
    eid = enquiry['id']
#     if enquiry['id'] == '11947313067':
#         if eid not in self.tracking_ids:
#             self.tracking_ids[eid] = {}
#             self.tracking_ids[eid]['datetime'] = pendulum.now()
#             self.tracking_ids[eid]['status'] = ['new']
#             self.save_tracking_ids()
#         return True
    
    if enquiry['responsible_person'].lower() != self.lname.lower():
        return False
    
    if eid in self.tracking_ids and self.tracking_ids[eid]['datetime'].diff().in_hours()<=12:
        if not enquiry['is_replied']:
            return True
    if len(enquiry['tags']) == 0 and enquiry['is_new']:
        if eid not in self.tracking_ids:
            self.tracking_ids[eid] = {}
            self.tracking_ids[eid]['datetime'] = pendulum.now()
            self.tracking_ids[eid]['status'] = ['new']
            self.save_tracking_ids()
        return True
    
    return False

inquiry.is_auto_reply_needed = types.MethodType(is_auto_reply_needed, inquiry)

In [ ]:
def check(self):
    self.load_url()
    inquiries = self.get_inquiries()

    for enquiry in inquiries:
        if not self.is_auto_reply_needed(enquiry):
            continue
        print('enquiry ' + enquiry['id'] + ' reply is needed')
        self.reply(enquiry)

inquiry.check = types.MethodType(check, inquiry)

In [ ]:
def reply(self, enquiry):
    try:
        self.open_inquiry_in_new_tab(enquiry)
        conversation = self.get_conversation()

        messages = conversation['messages']
        buyer = conversation['buyer']

        last_buyer_messages = []
        is_replied = False
        reply_count = 0
        for msg in messages:
            if msg['position'] == 'left':
                last_buyer_messages.append(msg)
                is_replied = False
            elif msg['position'] == 'right' and 'Seller Assistant' not in msg['user']:
                last_buyer_messages = []
                is_replied = True
                reply_count += 1

        eid = enquiry['id']
        tracking = self.tracking_ids[eid]
        last_status = tracking['status'][-1]
        if last_status == 'new':
            greetings = '<p>Pleased to hear from you.</p><br>'
            if buyer['email']:
                account = {}
                account['lid'] = self.lid
                account['lpwd'] = self.lpwd
                account['lname'] = self.lname
                emails = [buyer['email']]
                mime_message = Email.message_of_product_catalog(self.market, account, buyer['name'])
                if Email.send(account, emails, mime_message):
                    params = {'buyer': buyer['name'], 'greetings':greetings, 'email': buyer['email'], 'sender':self.lname}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    self.send_message(message)
                    tracking['status'].append('catalog was sent by email')
                    self.save_tracking_ids()
            else:
                params = {'buyer': buyer['name'], 'greetings':greetings, 'sender':self.lname}
                message = self.reply_templates['acquire_email_address'].substitute(params)
                self.send_message(message)
                tracking['status'].append('wait for email address')
                self.save_tracking_ids()
        elif 'catalog was sent' in last_status:
            pass
        elif 'wait for email address' in last_status:
            text = ''
            for msg in last_buyer_messages:
                text = text + '\n\n ' + msg['content']
            emails = re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)
            print(emails)
            if emails:
                tracking['emails'] = emails
                self.save_tracking_ids()
                account = {}
                account['lid'] = self.lid
                account['lpwd'] = self.lpwd
                account['lname'] = self.lname
                mime_message = Email.message_of_product_catalog(self.market, account, buyer['name'])
                if Email.send(account, emails, mime_message):
                    greetings = ''
                    params = {'buyer': buyer['name'], 'greetings':greetings, 'email': ', '.join(emails), 'sender':self.lname}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    self.send_message(message)
                    tracking['status'].append('catalog was sent by email')
                    self.save_tracking_ids()
    except Exception as e:
        self.notify("danger", '回复 询盘 [' + enquiry['id'] + '] 时 发生错误! ' + str(e))
        traceback.print_exc()
    finally:
        self.close_inquiry_and_switch_back()

inquiry.reply = types.MethodType(reply, inquiry)

In [ ]:
def load_url(self):
    self.browser.get(self.api)
    div = WebDriverWait(self.browser, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.aui-loading')))
    WebDriverWait(self.browser, 15).until(EC.staleness_of(div))
inquiry.load_url = types.MethodType(load_url, inquiry)

In [4]:
def load_url(self):
    while True:
        try:
            if self.browser is None:
                self.logger.info('open browser and login')
                alibaba = Alibaba(self.lid, self.lpwd, None, None, None, headless=False)
                alibaba.login()
                self.browser = alibaba.browser
                self.alibaba = alibaba

            self.browser.get(self.api)
            if 'login.alibaba.com' in self.browser.current_url:
                self.logger.info('Was out, login again!')
                self.alibaba.login()
                self.browser.get(self.api)
                
            div = WebDriverWait(self.browser, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.aui-loading')))
            WebDriverWait(self.browser, 15).until(EC.staleness_of(div))

            # try to close all follow-me-popups
            while True:
                btn_close = self.browser.find_elements_by_css_selector('div.follow-me-close')
                if btn_close:
                    webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
                    # wait 1 second to see if new popup commes
                    self.browser.implicitly_wait(1)
                    continue
                else:
                    break
            break
        except WebDriverException as e:
            if 'chrome not reachable' in str(e):
                self.logger.info('Browser Window was closed! Try to open a new browser window.')
                self.browser = None
            continue
inquiry.load_url = types.MethodType(load_url, inquiry)

In [7]:
inquiry.load_url()
# inquiries = inquiry.get_inquiries()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [ ]:
enquiry = inquiries[0]
inquiry.open_inquiry_in_new_tab(enquiry)
conversation = inquiry.get_conversation()
inquiry.close_inquiry_and_switch_back()
messages = conversation['messages']
buyer = conversation['buyer']
messages

In [ ]:
buyer

In [ ]:
inquiry.load_tracking_ids()
inquiry.tracking_ids

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
# inquiry.close_inquiry_and_switch_back()
# inquiry.tracking_ids['11947313067']['status'] = ['new']
enquiry = inquiries[0]
inquiry.reply(enquiry)

In [ ]:
inquiry.tracking_ids

In [ ]:
','.join(['a', 'b', 'df'])

In [ ]:
div.click()

In [ ]:
def try_login_webww():
    btn_login = WebDriverWait(inquiry.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-iconbar')))
    btn_login.click()

In [ ]:
panel = inquiry.browser.find_element_by_css_selector('#webatm2-panel')

In [ ]:
inquiries[11]

In [ ]:
inquiry = Inquiry(market)
inquiry.check()

In [ ]:
inquiry.check()

In [ ]:
inquiry.tracking_ids

In [ ]:
inquiry.tracking_ids['11947313067']['status'] = ['new']
inquiry.tracking_ids

In [ ]:
text = messages[0]['content']

In [ ]:
re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)

In [ ]:
inquiry.browser.save_screenshot("screenshot.png")

In [ ]:
inquiry.load_reply_templates()

In [ ]:
inquiry.reply_templates

In [ ]:
date_str = pendulum.now().to_atom_string()
print(date_str)
date = pendulum.parse(date_str)
print(date)

In [ ]:
inquiry.tracking_ids['100697230812'] = {}
inquiry.tracking_ids['100697230812']['datetime'] = pendulum.now()
inquiry.tracking_ids['100697230812']['status'] = 'new'
# self.save_tracking_ids()

In [ ]:
inquiry.tracking_ids = {}

In [ ]:
inquiry.save_tracking_ids()

In [ ]:
inquiry.tracking_ids

In [ ]:
inquiry.load_tracking_ids()

In [ ]:
date.diff().in_hours()<=12

In [ ]:
inquiry.tracking_ids